In [1]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 2 # 2aB
nH = 4
atoms = ""
for i in range(nH):
    atoms += f"O {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit='bohr', spin=0, verbose=4)
mol.build()

mf = scf.UHF(mol)
mf.kernel()

mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)

# s1e = mf.get_ovlp()
# olp = mf.mo_coeff[0].T @ s1e @ mf.mo_coeff[1]
# sign = np.array(np.sign(olp.diagonal()), dtype=int)
# print('<A|B> sign: ', sign)
# if -1 not in sign:
#     mf.mo_coeff[1][:,1] = -mf.mo_coeff[1][:,1]
# olp = mf.mo_coeff[0].T @ s1e @ mf.mo_coeff[1]
# sign = np.array(np.sign(olp.diagonal()), dtype=int)
# print('new <A|B> sign: ', sign)

nfrozen = 2
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun Oct 26 20:11:47 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 32
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00000

-0.141769382663651

In [2]:
mf.mo_coeff[1] += 1e-3
mf.make_rdm1(mf.mo_coeff,mf.mo_occ)
mf.max_cycle = -1
mf.kernel()



******** <class 'pyscf.scf.uhf.UHF'> ********
method = UHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpqmypyzzn
max_memory 4000 MB (current use 183 MB)
number electrons alpha = 16  beta = 16
Set gradient conv threshold to 3.16228e-05
init E= -297.354930775129
  alpha nocc = 16  HOMO = -0.17708715708812  LUMO = 0.346254980634227
  beta  nocc = 16  HOMO = -0.17817877054548  LUMO = 0.347264334640722
converged SCF energy = -297.354930775129  <S^2> = 0.96714457  2S+1 = 2.2064855


-297.3549307751292

In [ ]:
# alpha beta mo overlap <A|B>
# the diagonal elmt <A_i|B_i>
# they are very different!
# to compensate for the s=2
# s1e = mf.get_ovlp()
# olp_ab = mf.mo_coeff[0].T @ s1e @ mf.mo_coeff[1]
# print(olp_ab.diagonal())

[ 2.84666081e-02 -2.58162602e-02  1.62222986e-03  9.44658328e-04
  9.84541072e-01 -9.72644714e-01 -9.87192344e-01  1.41087503e-03
  2.69656923e-03 -8.00764506e-01  8.39044588e-05  8.05367681e-01
  2.45228680e-04  5.27137598e-04  9.59279569e-04 -9.78311439e-05
  4.53281009e-04 -9.97186177e-01 -9.96227151e-01  9.99378451e-01]


In [ ]:
# eris = mycc.ao2mo(mycc.mo_coeff)
# eccs = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
# print(mf.e_tot)
# print(mf.e_tot+eccs)
# mycc.t1 = (mycc.t1[0]*10,mycc.t1[1]*10)
# eccs = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
# print(mf.e_tot+eccs)
# eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
# print(mf.e_tot+eccsd)

-2.0886923819477192
-2.088694274795574
-2.0888799973483128
-2.165477505313509


In [47]:
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 3,
            'seed': 2,
            'walker_type': 'uhf',
            'trial': 'uhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import pyscf_interface
# from ad_afqmc.ccsd_pt import ccsd_pt, sample_ccsd_pt
# t1 = 5 * mycc.t1
# mycc.t1 = [10*mycc.t1[0],10*mycc.t1[1]]
# ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
pyscf_interface.prep_afqmc(mf,options,norb_frozen=0,chol_cut=1e-8)

from ad_afqmc import config, mpi_jax
import time
from jax import random
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

from ad_afqmc.ccsd_pt import prop_unrestricted
import types

trial._build_measurement_intermediates = types.MethodType(
    prop_unrestricted._build_measurement_intermediates, trial)

prop._build_propagation_intermediates = types.MethodType(
    prop_unrestricted._build_propagation_intermediates, prop)

prop._apply_trotprop = types.MethodType(
    prop_unrestricted._apply_trotprop, prop)

import h5py
chol_file='FCIDUMP_chol'
with h5py.File(chol_file, "r") as fh5:
    [nelec, nmo, ms, nchol] = fh5["header"]
    h0 = jnp.array(fh5.get("energy_core"))
    h1 = jnp.array(fh5.get("hcore")).reshape(nmo, nmo)
    h1_mod = jnp.array(fh5.get("hcore_mod")).reshape(nmo, nmo)
    chol = jnp.array(fh5.get("chol")).reshape(-1, nmo, nmo)


#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (16, 16)
# Number of basis functions: 20
# Number of Cholesky vectors: 135
#
# Number of MPI ranks: 1
#
# norb: 20
# nelec: (16, 16)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 3
# seed: 2
# walker_type: uhf
# trial: uhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [77]:
from pyscf import mcscf
norb_frozen = 0
mc = mcscf.UCASSCF(
                mf, mol.nao - norb_frozen, mol.nelectron - 2 * norb_frozen
            )
nelec = mc.nelecas
mc.mo_coeff = mf.mo_coeff
h1e, enuc = mc.get_h1eff()

chol_cut = 1e-8
nbasis = mol.nao

_, chol_a, _, _ = pyscf_interface.generate_integrals(
    mol, mf.get_hcore(), mf.mo_coeff[0], chol_cut, DFbas=None
)
# _, chol_b, _, _ = pyscf_interface.generate_integrals(
#     mol, mf.get_hcore(), mf.mo_coeff[1], chol_cut, DFbas=None
# )
chol_a = chol_a.reshape((-1, nbasis, nbasis))
s1e = mf.get_ovlp()
a2b = mf.mo_coeff[1].T @ s1e @ mf.mo_coeff[0]
# if separate build chol might not be the same number
chol_b = np.einsum('pr,grs,sq->gpq',a2b,chol_a,a2b.T)

chol_a = chol_a[:, mc.ncore[0] : mc.ncore[0] + mc.ncas,
                mc.ncore[0] : mc.ncore[0] + mc.ncas]
chol_b = chol_b.reshape((-1, nbasis, nbasis))
chol_b = chol_b[:, mc.ncore[1] : mc.ncore[1] + mc.ncas, 
                mc.ncore[1] : mc.ncore[1] + mc.ncas]

ham_data["h1"] = jnp.array(h1e)
# ham_data["chol_a"] = chol_a.reshape(chol_a.shape[0], -1)
# ham_data["chol_b"] = chol_b.reshape(chol_b.shape[0], -1)
ham_data["chol"] = jnp.array(
    [chol_a.reshape(chol_a.shape[0], -1),
     chol_b.reshape(chol_b.shape[0], -1)])

(noccA, noccB) = trial.nelec
mo_A = np.eye(trial.norb)[:,:noccA]
mo_B = np.eye(trial.norb)[:,:noccB]
wave_data['mo_coeff'] = [mo_A, mo_B]
if 'rdm1' in wave_data:
    del wave_data['rdm1']

In [66]:
chol = jnp.array(
    [chol_a.reshape(chol_a.shape[0], -1),
     chol_b.reshape(chol_b.shape[0], -1)])

In [78]:
config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1

# ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
# ham_data = ham.build_propagation_intermediates(
#     ham_data, prop, trial, wave_data
# )

ham_data = trial._build_measurement_intermediates(ham_data, wave_data)
ham_data = prop._build_propagation_intermediates(
    ham_data, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

prop_data["key"] = random.PRNGKey(seed + rank)
print('init walker energy: ', prop_data['e_estimate'])
print('mf enegry: ', mf.e_tot)
# print('ccsd energy: ', mycc.e_tot)
print('err', mf.e_tot - prop_data['e_estimate'])

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init walker energy:  -297.3549307260507
mf enegry:  -297.3549307751292
err -4.907849415758392e-08


In [50]:
ham_data["chol"].shape

(2, 135, 400)

In [51]:
from jax import lax
@partial(jit, static_argnums=(0, 3, 4))
def _block_scan(sample,prop_data,ham_data,prop,trial,wave_data):
    """Block scan function. Propagation and energy calculation."""
    prop_data["key"], subkey = random.split(prop_data["key"])
    fields = random.normal(
        subkey,
        shape=(
            sample.n_prop_steps,
            prop.n_walkers,
            ham_data["chol"][0].shape[0],
        ),
    )
    _step_scan_wrapper = lambda x, y: sample._step_scan(
        x, y, ham_data, prop, trial, wave_data
    )
    prop_data, _ = lax.scan(_step_scan_wrapper, prop_data, fields)
    prop_data["n_killed_walkers"] += prop_data["weights"].size - jnp.count_nonzero(
        prop_data["weights"]
    )
    prop_data = prop.orthonormalize_walkers(prop_data)
    prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
    energy_samples = jnp.real(
        trial.calc_energy(prop_data["walkers"], ham_data, wave_data)
    )
    energy_samples = jnp.where(
        jnp.abs(energy_samples - prop_data["e_estimate"]) > jnp.sqrt(2.0 / prop.dt),
        prop_data["e_estimate"],
        energy_samples,
    )
    block_weight = jnp.sum(prop_data["weights"])
    block_energy = jnp.sum(energy_samples * prop_data["weights"]) / block_weight
    prop_data["pop_control_ene_shift"] = (
        0.9 * prop_data["pop_control_ene_shift"] + 0.1 * block_energy
    )
    return prop_data, (block_energy, block_weight)

In [53]:
prop_data["n_killed_walkers"] = 0
prop_data, (block_energy, block_weight) = \
    _block_scan(sampler,prop_data,ham_data,prop,trial,wave_data)

In [54]:
print(block_energy, block_weight)

-297.3942124502986 2.9953064100819473


In [12]:
@partial(jit, static_argnums=0)
def _build_measurement_intermediates(trial, ham_data: dict, wave_data: dict) -> dict:
    ham_data["h1"] = (
        ham_data["h1"].at[0].set((ham_data["h1"][0] + ham_data["h1"][0].T) / 2.0)
    )
    ham_data["h1"] = (
        ham_data["h1"].at[1].set((ham_data["h1"][1] + ham_data["h1"][1].T) / 2.0)
    )
    ham_data["rot_h1"] = [
        wave_data["mo_coeff"][0].T.conj() @ ham_data["h1"][0],
        wave_data["mo_coeff"][1].T.conj() @ ham_data["h1"][1],
    ]
    ham_data["rot_chol"] = [
        jnp.einsum(
            "pi,gij->gpj",
            wave_data["mo_coeff"][0].T.conj(),
            ham_data["chol"][0].reshape(-1, trial.norb, trial.norb),
        ),
        jnp.einsum(
            "pi,gij->gpj",
            wave_data["mo_coeff"][1].T.conj(),
            ham_data["chol"][1].reshape(-1, trial.norb, trial.norb),
        ),
    ]
    return ham_data

In [13]:
@partial(jit, static_argnums=0)
def _calc_energy(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
) -> jax.Array:
    h0, rot_h1, rot_chol = ham_data["h0"], ham_data["rot_h1"], ham_data["rot_chol"]
    ene0 = h0
    green_walker = trial._calc_green(walker_up, walker_dn, wave_data)
    ene1 = jnp.sum(green_walker[0] * rot_h1[0]) \
         + jnp.sum(green_walker[1] * rot_h1[1])   # alpha + beta
    f_up = jnp.einsum(
        "gij,jk->gik", rot_chol[0], green_walker[0].T, optimize="optimal"
    )
    f_dn = jnp.einsum(
        "gij,jk->gik", rot_chol[1], green_walker[1].T, optimize="optimal"
    )
    c_up = vmap(jnp.trace)(f_up)
    c_dn = vmap(jnp.trace)(f_dn)
    exc_up = jnp.sum(vmap(lambda x: x * x.T)(f_up))
    exc_dn = jnp.sum(vmap(lambda x: x * x.T)(f_dn))
    ene2 = (
        jnp.sum(c_up * c_up)
        + jnp.sum(c_dn * c_dn)
        + 2.0 * jnp.sum(c_up * c_dn)
        - exc_up
        - exc_dn
    ) / 2.0

    return ene2 + ene1 + ene0

In [14]:
ham_data = _build_measurement_intermediates(trial, ham_data, wave_data)

In [17]:
e_afqmc = _calc_energy(trial,mo_A,mo_B,ham_data,wave_data)
print(e_afqmc)
print(mf.e_tot-e_afqmc)

-297.36800815964307
-4.640344286599429e-08


In [ ]:
from jax import scipy as jsp
from jax import lax
@partial(jit, static_argnums=(0, 2))
def _build_propagation_intermediates(prop, ham_data, trial, wave_data):
    rdm1 = wave_data["rdm1"]
    mf_shift_a = 1.0j * vmap(
        lambda x: jnp.sum(x.reshape(trial.norb, trial.norb)
                          * rdm1[0]))(ham_data["chol"][0])
    mf_shift_b = 1.0j * vmap(
        lambda x: jnp.sum(x.reshape(trial.norb, trial.norb) 
                          * rdm1[1]))(ham_data["chol"][1])
    ham_data["mf_shifts"] = mf_shift_a + mf_shift_b
    ham_data["h0_prop"] = (
        - ham_data["h0"] - jnp.sum(ham_data["mf_shifts"]**2) / 2.0
                                   )
    # alpha v_prrq
    v0_a = 0.5 * jnp.einsum(
        "gik,gjk->ij",
        ham_data["chol"][0].reshape(-1, trial.norb, trial.norb),
        ham_data["chol"][0].reshape(-1, trial.norb, trial.norb),
        optimize="optimal",
    )
    # beta  v_prrq
    v0_b = 0.5 * jnp.einsum(
        "gik,gjk->ij",
        ham_data["chol"][1].reshape(-1, trial.norb, trial.norb),
        ham_data["chol"][1].reshape(-1, trial.norb, trial.norb),
        optimize="optimal",
    )
    # alpha
    v1_a = jnp.real(
        1.0j
        * jnp.einsum(
            "g,gik->ik",ham_data["mf_shifts"],
            ham_data["chol"][0].reshape(-1, trial.norb, trial.norb),
        )
    )
    # beta
    v1_b = jnp.real(
        1.0j
        * jnp.einsum(
            "g,gik->ik",ham_data["mf_shifts"],
            ham_data["chol"][1].reshape(-1, trial.norb, trial.norb),
        )
    )
    h1_mod = ham_data["h1"] - jnp.array([v0_a + v1_a, v0_b + v1_b])
    ham_data["exp_h1"] = jnp.array(
        [
            jsp.linalg.expm(-prop.dt * h1_mod[0] / 2.0),
            jsp.linalg.expm(-prop.dt * h1_mod[1] / 2.0),
        ]
    )
    return ham_data

In [29]:
@partial(jit, static_argnums=(0,))
def _apply_trotprop(
    prop, ham_data, walkers, fields
):
    n_walkers = walkers[0].shape[0]
    batch_size = n_walkers // prop.n_batch

    def scanned_fun(carry, batch):
        field_batch, walker_batch_0, walker_batch_1 = batch
        # alpha
        vhs_a = (
            1.0j
            * jnp.sqrt(prop.dt)
            * field_batch.dot(ham_data["chol"][0]).reshape(
                batch_size, walkers[0].shape[1], walkers[0].shape[1]
            )
        )
        # beta
        vhs_b = (
            1.0j
            * jnp.sqrt(prop.dt)
            * field_batch.dot(ham_data["chol"][1]).reshape(
                batch_size, walkers[0].shape[1], walkers[0].shape[1]
            )
        )
        walkers_new_0 = vmap(prop._apply_trotprop_det, in_axes=(None, 0, 0))(
            ham_data["exp_h1"][0], vhs_a, walker_batch_0
        )
        walkers_new_1 = vmap(prop._apply_trotprop_det, in_axes=(None, 0, 0))(
            ham_data["exp_h1"][1], vhs_b, walker_batch_1
        )
        return carry, [walkers_new_0, walkers_new_1]

    _, walkers_new = lax.scan(
        scanned_fun,
        None,
        (
            fields.reshape(prop.n_batch, batch_size, -1),
            walkers[0].reshape(
                prop.n_batch, batch_size, walkers[0].shape[1], walkers[0].shape[2]
            ),
            walkers[1].reshape(
                prop.n_batch, batch_size, walkers[1].shape[1], walkers[1].shape[2]
            ),
        ),
    )
    walkers = [
        walkers_new[0].reshape(n_walkers, walkers[0].shape[1], walkers[0].shape[2]),
        walkers_new[1].reshape(n_walkers, walkers[1].shape[1], walkers[1].shape[2]),
    ]
    return walkers